# Counterfactual Instance

**Counterfactual Instance(CFI)** 는 "어떤 사건 $X$가 일어나지 않았더라면 $Y$가 일어나지 않았을 텐데"에서 조건 '$X$가 일어나지 않았더라면'에 해당한다. 머신러닝 관점에서 $X$를 설명변수 $x$로, $Y$를 예측 레이블 $\hat{y}$로 대체하면, 앞의 문장은  
$$
x를\ x'으로\ 변경하면\ 예측\ 레이블이\ \hat{y}와\ 다른\ \hat{y}'으로\ 변경되었을\ 텐데
$$

로 변경할 수 있다. 여기서 $x'$이 CFI가 되며, 이 $x'$은 $x$를 최소한으로 변경하여(Sparse) 구하려고 한다. 좀 더 구체적으로 CFI를 정리하면 다음과 같다.

1. CFI $x'$의 예측 레이블은 $x$의 예측 레이블과 다르지만  

2. $x$와 $x'$의 차이는 **사람이 해석가능한(human-Intepretable)** 차이여야 한다.

여기서 **human-Intepretable**의 의미는, 통계적인 의미로는 $x$와 $x'$가 같은 분포여야 한다는 의미이다. 둘이 동일한 분포여야만 $x'$이 실현가능하기 때문이다. 예를 들어, 어떤 여성의 대출 신청이 거부되었는데, 제시된 CFI로 gender를 남성으로 바꾸라는 것이면 이는 실현 불가능한 경우에 해당한다. 이미지의 경우, 사람의 눈으로 $x$와 $x'$을 식별 불가능할 정도로 미세한 차이여야 한다는 뜻이다.

$x$와 $x'$을 각각 주어진 설명변수와 해당 설명변수로부터 생성한 CFI라고 할 때, $x'$에 대한 손실함수는
$$
L(x')=(f(x')-y')^2+ \lambda d(x,x')
$$
로 정의하여 해당 손실함수를 최소화하는 $x'$을 경사하강법으로 구하는 방법을 제시했다. 여기서 $\lambda$는 hyperparameter, $f(x')$은 counterfactual을 입력하여 구한 예측 레이블 확률, $y'$은 CFI로 변경되길 원하는 레이블(또는 레이블 확률)이다.

설명변수 개수가 $p$개라고 가정할 때
$$
d(x,x')=\sum_{j=1}^{p}\frac{\left| x_j-x_j' \right|}{MAD_j}
$$
$$
MAD_j=median_i =(\left| x_{ij}-median_i(x_{ij}) \right|),\ where\ x_{ij}\ is\ j-th\ feature\ of\ i-th\ sample
$$

로 정의할 수 있다. MAD는 ***mean absolute deviation***의 약자이다. 이러한 $d(x,x')$은 설명변수의 scale에 의존하지 않으며 $x$와 $x'$의 차이를 최소화하는 sparse를 만족시키기 위해 $L_1-norm$을 사용한다. sparse를 만족시키기 위해 손실함수 $L(x')$의 $\lambda$를 가능한 한 큰 값으로 설정한다.  
즉, 손실함수 $L(x')$에서
$$
min_{x'}max_{\lambda}L(x'|x)
$$
으로 CFI $x'$을 구하게 된다. 최대 $\lambda$를 구학 위해 주어진 허용오차 $\epsilon$에 대해
$$
\left| f(x')-y' \right| \lt \epsilon
$$
를 만족하는 조건에서 최대의 $\lambda$를 구한 후, $x'$을 구하게 된다.

CFI는 블랙박스 모형뿐만 아니라 내부구조(모수와 은닉층의 연결구조를 포함하여)를 알고 있는 유리박스 모형에도 적용가능하다. `Tensorflow-keras`를 이용한 모든 딥러닝 모형은 tensorflow의 자동미분기능 때문에 CFI 관점에서는 유리박스 모형에 해당된다. 입력과 출력만 관측되는 블랙박스 모형에서는 수치해석 미분(numerical differentiation)으로 경사하강법을 적용하여 손실함수를 최소화한다.